In [ ]:
#=========================================================================
# EXAMPLE ON FITTING: CO2 curve
#=========================================================================
import os,warnings
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.dates import YearLocator,DateFormatter
from matplotlib.ticker import MultipleLocator


In [ ]:
#=========================================================================
# Load the Keeling Curve.
#=========================================================================
fileName = 'co2_mm_mlo.txt'
count = 0
with open(fileName,'r') as f:
    for line in f:
        if line.startswith('#'):
            count+=1
            
CO2 = {}
CO2['year'],CO2['month'],CO2['co2'] = np.loadtxt(fileName,skiprows=count,usecols=(0,1,4),unpack=True)

#CO2 = LoadMATFile('CO2.mat')


In [ ]:
plt.figure()
plt.plot(CO2['co2'])
plt.show(block=False)

In [ ]:
#=========================================================================
# Eliminate any nan values.
#=========================================================================
dates = np.array([datetime(int(y),int(m),15) for y,m in zip(CO2['year'],CO2['month'])])

y = CO2['co2'].copy()
t = np.arange(1,CO2['co2'].size+1)
N = len(t)

print(dates)

In [ ]:
#=========================================================================
# Choose an observational/measurement error. Make error covariance Rnn
#=========================================================================
std_n=2;
#std_n=0.1

Rnn = std_n**2*np.diag(np.ones(N))


In [ ]:
#=========================================================================
# Set the matrix E for the FULL model y = a + b*t + c*t^2 + d*cos(2*pi*t/12) + e*sin(2*pi*t/12)
#=========================================================================
E = np.ones((N,5))*np.nan

E[:,0] = 1
E[:,1] = t
E[:,2] = t**2
E[:,3] = np.cos(2.*np.pi*t/12.)
E[:,4] = np.sin(2.*np.pi*t/12.);


In [ ]:
#=========================================================================
# Set the parameters we will want to test. Can be a subset of the columns.
#=========================================================================
params = np.arange(2) #Linear fit.
#params = np.arange(3) #Linear + quadratic.
#params =np.arange(5) # Full model.

E = E[:,params]
print(E)


In [ ]:
#=========================================================================
# Run the model to get xhat.
#=========================================================================
invETE = np.linalg.inv(np.dot(E.T,E))
xhat = invETE.dot(E.T).dot(y)
yhat = np.dot(E,xhat)

print(xhat)


In [ ]:
#=========================================================================
# Uncertainty in the model parameters.
#=========================================================================
P = invETE.dot(E.T).dot(Rnn).dot(E).dot(invETE)

se = np.sqrt(np.diag(P))
R2 = np.nanvar(yhat-np.nanmean(y))/np.nanvar(y)

nhat = y - yhat
print(se)
print(R2)

In [ ]:
#=========================================================================
# Plot results.
#=========================================================================
years = YearLocator(5)   
yearsFmt = DateFormatter('%Y')

fig,axes = plt.subplots(figsize=(12,8),nrows=2,ncols=2,sharex=True)

ax1 = axes[0,0]; ax2 = axes[0,1]; ax3 = axes[1,0]; ax4 = axes[1,1]

#FIT
ax1.plot(dates,y,'k-',lw=2)
ax1.plot(dates,yhat,'r-',lw=2)
ax1.set_title('LSQ FIT (Obs-Black, Estimate-Red)',name='Calibri',size=14,weight='bold')
ax1.set_ylabel('PPM',name='Calibri',size=12,weight='bold')
ax1.set(ylim=[300,420],yticks=np.arange(300,430,20),xlim=[datetime(1970,1,15),datetime(2019,1,15)])

ax1.xaxis.set_major_locator(YearLocator(5))
ax1.xaxis.set_minor_locator(YearLocator(1))
ax1.xaxis.set_major_formatter(yearsFmt)

ax1.yaxis.set_minor_locator(MultipleLocator(2))

#TREND
trend = np.dot(E[:,:3],xhat[:3])
ax2.plot(dates,trend,'k-',lw=2)
ax2.set_title('TREND',name='Calibri',size=14,weight='bold')
ax2.set(ylim=[300,420],yticks=np.arange(300,431,20))
ax2.yaxis.set_minor_locator(MultipleLocator(2))


#RESIDUALS
ax3.plot(dates,nhat,'k-',lw=2)
ax3.set_title('RESIDUALS',name='Calibri',size=14,weight='bold')
ax3.set(ylim=[-5,5],yticks=np.arange(-5,5.1))
ax3.set_ylabel('PPM',name='Calibri',size=12,weight='bold')
ax3.set_xlabel('Year',name='Calibri',size=12,weight='bold')
ax3.yaxis.set_minor_locator(MultipleLocator(0.2))

if E.shape[1] == 5:
    seasonal = np.dot(E[:,3:],xhat[3:])
    ax4.plot(dates,seasonal,'k-',lw=2)
    ax4.set_title('SEASONAL',name='Calibri',size=14,weight='bold')
    ax4.set(ylim=[-3,3],yticks=np.arange(-3,3.1,0.5))
    ax4.set_xlabel('Year',name='Calibri',size=12,weight='bold')
    ax4.yaxis.set_minor_locator(MultipleLocator(0.1))

fig.autofmt_xdate()
plt.savefig(F'CO2_Fit_NPARAMS_{len(params)}.png',dpi=300)
plt.show(block=False)
